In [233]:
import copy
import pandas as pd
import random
import re
import sys

inputFile = pd.read_csv(u"Управление анкетой 2.csv", header=-1, sep='\t')
inputFile = inputFile.fillna('')
lengthOfInputFile = inputFile.shape[0]
widthOfInputFile = inputFile.shape[1]
N = 1000   # Количество опрашиваемых

In [234]:
#inputFile

In [235]:
#inputFile[0][3]

# Вспомогательные функции

## Добавление распределения

In [236]:
def addDistribution(distributionsDict, inputFile, iVariants, iValues, title, distributionName, decodingDict):
    j = 0
    distributionsDict[title][distributionName] = []
    while True:
        #print 'строка ' + str(iValues) + '\tстолбец ' + str(j)
        if j >= inputFile.shape[1]:
            #print 'Край таблицы'
            break
        if inputFile[j][iVariants] == '':
            #print 'Кончились варианты'
            break
        if inputFile[j][iValues] == '':
            print 'Отсутствует вероятность для ' + inputFile[j][iVariants] + ' в строке ' + str(iValues)
            return -1
        decodingDict[title][j+1] = inputFile[j][iVariants]     # Создаём заодно таблицу дешифровки численных кодов
        distributionsDict[title][distributionName].append(float(inputFile[j][iValues]))
        j += 1
    
    # Проверка
    mySum = 0
    for item in distributionsDict[title][distributionName]:
        mySum += item
    if mySum > 1:
        print 'Неправильно указаны вероятности для вопроса ' + str(title) + ' в строке ' + str(iValues)
        return -1
    
    #print pd.DataFrame(distributionsDict[title][distributionName])
    return 0

## Добавление ответа

In [237]:
def addAnswer(answersDict, title, distribution):
    answer = random.random()
    i = 1
    mySum = 0
    for item in distribution:
        #print item
        #print distribution[item]
        mySum += item
        #print mySum
        if answer < mySum:
            #print 'answer ' + str(answer) + '\t<\tmySum ' + str(mySum)
            answer = i
            #print 'Final answer:\t' + str(answer)
            break
        #else:
            #print 'answer ' + str(answer) + '\t>\tmySum ' + str(mySum)
        i += 1
    #print answer
    answersDict[title].append(answer)
    return 0

# Создаём ключи словаря

In [238]:
questionsDict = {}
for i in range(1, 23):
    questionsDict['q' + str(i)] = {}

# Заполняем правила

In [239]:
distributionsDict = copy.deepcopy(questionsDict)
decodingDict = copy.deepcopy(questionsDict)
questionNumber = 0
for i in range(lengthOfInputFile):  # Идём по каждой строке исходной таблицы
    firstCell = inputFile[0][i]
    if firstCell.lstrip('#') == str(questionNumber + 1):    # Нашли что-то похожее на вопрос?
        questionNumber += 1
        
        if questionNumber == 21:
            continue
        
        # Здесь надо будет научиться определять тип вопроса
        
        # Выбор одного из вариантов
        j = 0
        EndOfQuestion = False
        while not EndOfQuestion:
            errorCode = 0
            
            if i + j >= lengthOfInputFile:    # Обработка конца таблицы
                EndOfQuestion = True
                continue
                
            firstCell = inputFile[0][i + j]    # Новый вопрос знаменуется строкой с решёткой в начале
            if firstCell.lstrip('#') == str(questionNumber + 1):
                EndOfQuestion = True
                continue
            
            if j == 2:    # Распределение по умолчанию
                errorCode = addDistribution(distributionsDict, inputFile, i + j, i + j + 1, 'q' + str(questionNumber), 'Default', decodingDict)
            
            if firstCell.find('&&&') != -1: # Распределение с логическим условием
                k = 1
                key = ['&&&']
                while inputFile[k][i + j] != '':
                    key.append( (inputFile[k][i + j], inputFile[k + 1][i + j]) ) 
                    if k+1 == widthOfInputFile:
                        break
                    k += 2
                key = tuple(key)
                
                errorCode = addDistribution(distributionsDict, inputFile, i + 2, i + j + 1, 'q' + str(questionNumber), key, decodingDict)
            
            if ( errorCode == -1 ):
                print 'Поправьте распределение!'
                sys.exit()
                
            #print questionNumber
                
               
            j += 1
        
        questionsDict['q' + str(questionNumber)]

In [240]:
#pd.DataFrame(distributionsDict)

In [241]:
#decodingDict

# Формируем заголовки таблицы ответов

In [242]:
answersDict = copy.deepcopy(questionsDict)
for i in range(1, 23):
    title = 'q' + str(i)
    answersDict[title] = []

# Бегаем по опрашиваемым

In [243]:
for i in range(N):
    questionNumber = 0
    for j in range(1, 23):
        
        title = 'q' + str(j)
        
        
        if j == 21:    #заглушка
            answersDict[title].append(0)
            continue
            
        key = 'Default'
        for distribution in distributionsDict[title]:
            if distribution[0] == '&&&':    # Проверим-ка, выполняется ли условие:
                # Если ответ на вопрос ## отвечает значению ???????, то выбираем это распределение
                # Проблема: мы только что перешли от тройного словаря к двойному словарю со списками, 
                # и пар номера-значения у нас больше нет
                targetQuestionTitle = 'q'+str(distribution[1][0])
                #print decodingDict[targetQuestionTitle][answersDict[targetQuestionTitle][-1]]
                #print str(distribution[1][1])
                if decodingDict[targetQuestionTitle][answersDict[targetQuestionTitle][-1]] == str(distribution[1][1]):
                    key = distribution
        addAnswer(answersDict, title, distributionsDict[title][key])  

In [244]:
distribution

'Default'

In [245]:
result = pd.DataFrame(answersDict)


In [246]:
#result.columns

In [247]:
#sorted(sorted(result.columns), key = lambda myStr: len(myStr))

In [248]:
result = result.reindex(sorted(sorted(result.columns), key = lambda myStr: len(myStr)), axis=1)

In [249]:
result.to_csv('output.csv', sep='\t', header=True, index=False)

In [250]:
result

,q1,q2,q3,q4,q5,q6,q7,q8,q9,q10,...,q13,q14,q15,q16,q17,q18,q19,q20,q21,q22
0,1,2,1,2,2,1,2,1,2,1,...,1,3,2,3,2,2,2,1,0,1
1,3,3,1,1,1,1,1,1,2,1,...,2,2,2,1,1,2,2,2,0,1
2,2,2,3,1,2,1,1,2,1,2,...,1,1,1,2,2,2,2,1,0,2
3,2,1,3,1,1,2,1,1,2,1,...,1,1,1,1,1,3,1,1,0,2
4,1,2,3,1,3,1,1,1,2,1,...,2,1,1,1,1,1,1,2,0,1
5,1,3,1,3,2,1,2,1,1,1,...,2,3,1,2,2,1,1,3,0,2
6,1,1,3,1,1,1,3,2,2,2,...,2,2,1,1,2,1,1,1,0,2
7,1,1,3,3,3,1,2,3,1,1,...,2,1,1,2,1,3,1,1,0,2
8,2,2,1,1,1,2,2,1,3,1,...,1,1,2,1,1,2,3,1,0,3
9,1,2,1,2,1,3,1,3,3,1,...,1,1,3,1,2,1,3,1,0,1
